<a href="https://colab.research.google.com/github/albanda/CE888/blob/master/lab4-recommender/rec_latent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jokes Rating

Load required libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)

Now load the data

In [4]:
data = pd.read_csv("https://raw.githubusercontent.com/PauMarquez99/DataScience-CE888/lab6/lab6/jester-data-1.csv", header=None)

In [5]:
data.shape

(24983, 101)

In [34]:
# Drop column, it is only the number of jokes rated by the user
# Replace value for NAN as it does not represent a rating
df = data.replace(99.00, np.NaN)
df = df.iloc[: , 1:]
df

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,NaN,NaN,NaN,NaN,-7.77,NaN,6.70,-6.75,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24981,NaN,NaN,NaN,NaN,-9.71,NaN,4.56,-8.30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df = df.fillna(df.mean())
df

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-7.820000,8.79000,-9.660000,-8.160000,-7.52,-8.500000,-9.85,4.17,-8.980000,-4.760000,...,2.820000,1.179744,2.504041,1.173482,1.03913,1.528003,-5.63000,0.767692,-0.031244,1.354711
1,4.080000,-0.29000,6.360000,4.370000,-2.38,-9.660000,-0.73,-5.34,8.880000,9.220000,...,2.820000,-4.950000,-0.290000,7.860000,-0.19000,-2.140000,3.06000,0.340000,-4.320000,1.070000
2,0.904573,0.20846,0.316574,-1.449114,9.03,9.270000,9.03,9.27,-0.557031,1.301231,...,2.038527,1.179744,2.504041,9.080000,1.03913,1.528003,1.67375,0.767692,-0.031244,1.354711
3,0.904573,8.35000,0.316574,-1.449114,1.80,8.160000,-2.82,6.21,-0.557031,1.840000,...,2.038527,1.179744,2.504041,0.530000,1.03913,1.528003,1.67375,0.767692,-0.031244,1.354711
4,8.500000,4.61000,-4.170000,-5.390000,1.36,1.600000,7.04,4.61,-0.440000,5.730000,...,5.190000,5.580000,4.270000,5.190000,5.73000,1.550000,3.11000,6.550000,1.800000,1.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.440000,7.43000,9.080000,2.330000,3.20,6.750000,-8.79,-0.53,-8.740000,7.230000,...,8.830000,-1.210000,9.220000,-6.700000,8.45000,9.030000,6.55000,8.690000,8.790000,7.430000
24979,9.130000,-8.16000,8.590000,9.080000,0.87,-8.930000,-3.50,5.78,-8.110000,4.900000,...,-1.170000,-5.730000,-1.460000,0.240000,9.22000,-8.200000,-7.23000,-8.590000,9.130000,8.450000
24980,0.904573,0.20846,0.316574,-1.449114,-7.77,1.594447,6.70,-6.75,-0.557031,1.301231,...,2.038527,1.179744,2.504041,1.173482,1.03913,1.528003,1.67375,0.767692,-0.031244,1.354711
24981,0.904573,0.20846,0.316574,-1.449114,-9.71,1.594447,4.56,-8.30,-0.557031,1.301231,...,2.038527,1.179744,2.504041,1.173482,1.03913,1.528003,1.67375,0.767692,-0.031244,1.354711


In [36]:
means = df.mean(axis=0)
print('Best Joke Index: ', means.idxmax(), '  Rating: ', means.max())
print('Worst Joke Index: ', means.idxmin(), '  Rating: ', means.min())

Best Joke Index:  50   Rating:  3.6650848950824924
Worst Joke Index:  58   Rating:  -3.8338796373689963


In [29]:
# In the jokes dataset, what's the rating that user 19947 has given to joke 58?
df.iloc[7232 , 19]

8.79

## Helper Function

In [38]:
def replace_m(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(orig.values!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])


valset, idx = replace_m(df.values, 0.1)

AttributeError: 'numpy.ndarray' object has no attribute 'values'